### Прочитали fasta формат

In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("sequence.fasta", format="fasta")


In [37]:
record.seq[0:127]

Seq('MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYET...DIT', SingleLetterAlphabet())

### Рассмотрим результат BLAST поиска

In [173]:
result_handle = open('W8ZR3WEN015-Alignment.xml')

In [174]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

In [175]:
good_almns = {}
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            titles = alignment.title.split('>')
            for title in titles:
                t = title.split('|')
                ss =['517848989']
                for s in ss:
                    if t[1] == s:
                        print("****Alignment****")
                        print("sequence:", alignment.title)
               


Варианты, которые получлсь у меня и совпали со статьей 504822023, 505222758, 

In [50]:
good_almns

{'gi|299008122|gb|ADJ00051.1| Evoglow [Promoter probe vector pEvoGlowRed] >gi|299008129|gb|ADJ00057.1| Evoglow [Mariner mini-transposon delivery vector pMaGlowRedKm] >gi|299008135|gb|ADJ00062.1| Evoglow [Mariner mini-transposon delivery vector pMaTcGlow] >gi|299008142|gb|ADJ00067.1| Evoglow [Mariner mini-transposon delivery vector pMaGlow] >gi|299008157|gb|ADJ00079.1| Evoglow [Reporter vector pGlowRed]': 'MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNARFLQGKHTDPAEVDNIRTALQNKEPVTVQIQNYKKDGTMFWNELNIDPMEIEDKTYFVGIQNDIT',
 'gi|126023750|gb|ABN71355.1| FMN-based fluorescence protein, partial [synthetic construct] >gi|238480383|gb|ACR43777.1| FMN-based fluorescent protein CaFbFP [synthetic construct]': 'MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNARFLQGKHTDPAEVDNIRTALQNKEPVTVQIQNYKKDGTMFWNELNIDPMEIEDKTYFVGIQNDIT',
 'gi|1061250688|gb|AOH95442.1| Evoglow [Cloning vector pPBAsyn]': 'MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNARFLQGKHTDPAEVDNIRTA

In [45]:
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print(hsp.query_start, hsp.query_end, sep = ' ')
            print(hsp.query[0:75] + '...')

****Alignment****
sequence: gi|931147283|gb|ALG35680.1| YtvA-LOV, partial [synthetic construct]
1 127
MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEV...
****Alignment****
sequence: gi|931147291|gb|ALG35684.1| YtvA-LOV, partial [synthetic construct]
1 127
MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEV...
****Alignment****
sequence: gi|1242725039|ref|WP_095843885.1| blue-light photoreceptor [Bacillus subtilis] >gi|1241297661|gb|ASZ62492.1| biphenyl 2,3-dioxygenase [Bacillus subtilis] >gi|1321401427|gb|PLV38306.1| Blue-light photoreceptor [Bacillus subtilis subsp. subtilis]
1 127
MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEV...
****Alignment****
sequence: gi|490533850|ref|WP_004399022.1| MULTISPECIES: blue-light photoreceptor [Bacillus] >gi|16080086|ref|NP_390912.1| blue light GTP-binding receptor [Bacillus subtilis subsp. subtilis str. 168] >gi|20138874|sp|O34627.1|PHOT_BACSU RecName: Full=Blue-light p

sequence: gi|1024740025|gb|KZZ83090.1| hypothetical protein AS29_020095 [Bacillus sp. SJS]
14 127
EVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPV...
****Alignment****
sequence: gi|1222176780|ref|WP_090231894.1| PAS domain-containing protein [Fictibacillus solisalsi]
12 127
QLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKE...
****Alignment****
sequence: gi|516042645|ref|WP_017473228.1| PAS domain-containing protein [Amphibacillus jilinensis]
15 127
VIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVT...
****Alignment****
sequence: gi|1176702879|ref|WP_082529813.1| PAS domain-containing protein [Bacillus sp. Leaf406]
15 127
VIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVT...
****Alignment****
sequence: gi|1011869840|ref|WP_062687197.1| MULTISPECIES: PAS domain-containing protein [Bacillus] >gi|959356422|gb|KSU86687.1| biphenyl 2,3-dioxygenase [Bacillus sp. SGD-V-76] >gi|1049610592|emb|

24 127
RVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVTVQIQNYKKD...
****Alignment****
sequence: gi|1180682253|ref|WP_084167433.1| PAS domain-containing protein [Bacillus lehensis]
19 127
ALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVTVQIQ...
****Alignment****
sequence: gi|1238394516|ref|WP_095219685.1| PAS domain-containing protein [Terribacillus saccharophilus] >gi|1237189775|gb|PAD34858.1| hypothetical protein CHH56_12405 [Terribacillus saccharophilus] >gi|1237252399|gb|PAD95606.1| hypothetical protein CHH50_12640 [Terribacillus saccharophilus] >gi|1237256042|gb|PAD99184.1| hypothetical protein CHH48_13535 [Terribacillus saccharophilus]
17 127
KKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVTVQ...
****Alignment****
sequence: gi|916603466|ref|WP_051210557.1| PAS domain-containing protein [Sporosarcina ureae] >gi|1174583569|gb|ARF13897.1| hypothetical protein SporoS204_06910 [Sporosarcina ureae]
24 127
RVGVVITDPALEDNPIVY

sequence: gi|909691516|ref|WP_049934764.1| PAS domain-containing protein [Haloplanus natans]
13 127
LEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEP...
****Alignment****
sequence: gi|1371238036|gb|PSP70722.1| hypothetical protein BRC80_08015 [Halobacteriales archaeon QH_9_66_26]
12 127
QLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKE...
****Alignment****
sequence: gi|495936871|ref|WP_008661450.1| PAS domain S-box protein [Rhodopirellula europaea] >gi|448883278|gb|EMB13815.1| signal transduction histidine kinase with CheB and CheR activity [Rhodopirellula europaea 6C]
12 127
QLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKE...
****Alignment****
sequence: gi|916723907|ref|WP_051330998.1| PAS domain-containing protein [Aneurinibacillus terranovensis]
19 127
ALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVTVQIQ...
****Alignment****
sequence: gi|919369432|ref|WP_052812346.1| PAS domain S-bo

sequence: gi|313634231|gb|EFS00866.1| blue-light photoreceptor [Listeria seeligeri FSL N1-067]
12 127
QLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKE...
****Alignment****
sequence: gi|516870416|ref|WP_018139391.1| MULTISPECIES: PAS sensor domain-containing protein [Thioalkalivibrio]
14 127
EVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPV...
****Alignment****
sequence: gi|947716783|ref|WP_056379114.1| PAS domain S-box protein [Methylobacterium sp. Leaf465] >gi|945326285|gb|KQT83725.1| histidine kinase [Methylobacterium sp. Leaf465]
19 127
ALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPVTVQIQ...
****Alignment****
sequence: gi|517781515|ref|WP_018951723.1| PAS sensor domain-containing protein [Thioalkalivibrio sp. AKL12]
14 127
EVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYETEEILGKNCRFLQGKHTDPAEVDNIRTALQNKEPV...
****Alignment****
sequence: gi|836665709|ref|WP_047813681.1| PAS domain S-box protein [Rhodopirellula islandi

### Скачивание OneKP

In [4]:
from bs4 import BeautifulSoup   # Для обработки HTML
import requests # для получения кода страницы

In [5]:
data = requests.get('http://www.onekp.com/public_data.html')

In [9]:
soup = BeautifulSoup(data.text, 'html.parser')

In [1]:
print(soup.prettify())

Получим все ссылки

In [28]:
s= []
for string in soup.find_all('a'):
    s.append(string.get('href'))

Будем обращаться для скачивания по необходимым ссылкам

In [57]:
import urllib

In [55]:
res = [elem if elem.find("Trans-assembly")!=-1 else None for elem in s]

In [59]:
url = 'http://206.12.96.204/onekp/URDJ-SOAPdenovo-Trans-assembly.fa.bz2'
urllib.request.urlretrieve(url, "URDJ.bz2")

('URDJ.bz2', <http.client.HTTPMessage at 0x7f476baf3e10>)

### Разделение файла с белками из Interpro

In [5]:
with open('proteins-matched_interpro.txt', 'r') as file:
    n = 0
    s_1, s_2, s_3, s_4 = [], [], [], []
    for line in file:
        if line.startswith('>'):
            n += 1
        if n <= 98328:
            s_1.append(line)
        elif 98328 < n <= 98328*2:
            s_2.append(line)
        elif 98328 * 2 < n <= 98328*3:
            s_3.append(line)
        else:
            s_4.append(line)
            
            

In [23]:
with open('protein_1.txt','w') as file:
    file.write(''.join(s_1))
with open('protein_2.txt','w') as file:
    file.write(''.join(s_2))
with open('protein_3.txt','w') as file:
    file.write(''.join(s_3))
with open('protein_4.txt','w') as file:
    file.write(''.join(s_4))

### Парсинг файлов mast и meme 

In [58]:
from bs4 import BeautifulSoup
with open("Without_Cys/mast_interpro_3_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')
#print(soup.prettify())

In [59]:
motifs = soup.motifs.findAll('motif')

In [60]:
length = [motif['length'] for motif in motifs]

In [61]:
length

['35', '31', '29']

На 40 месте  в мотиве стоит **С** на который и нужно проверить 

In [62]:
sequences = soup.sequences.findAll('sequence')

In [63]:
import re
reg = re.compile('[^a-zA-Z ]')

Нашли все имена таких сиквенсов и также запсали данные о них в формати xml

In [64]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [65]:
with open('Without_Cys/Cysless_sequences_3.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

In [66]:
len(Cys_seq_names)

839

In [3]:
with open('pfam.txt', 'r') as f:
    text = f.readlines()

In [4]:
text

['# pfam_scan.pl,  run at Wed Nov  7 20:55:29 2018\n',
 '#\n',
 '# Copyright (c) 2009 Genome Research Ltd\n',
 '# Freely distributed under the GNU \n',
 '# General Public License\n',
 '#\n',
 '# Authors: Jaina Mistry (jaina@ebi.ac.uk), \n',
 '#          Rob Finn (rdf@ebi.ac.uk)\n',
 '#\n',
 '# This is free software; you can redistribute it and/or modify it under\n',
 '# the terms of the GNU General Public License as published by the Free Software\n',
 '# Foundation; either version 2 of the License, or (at your option) any later version.\n',
 '# This program is distributed in the hope that it will be useful, but WITHOUT\n',
 '# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS\n',
 '# FOR A PARTICULAR PURPOSE. See the GNU General Public License for more\n',
 '# details.\n',
 '#\n',
 '# You should have received a copy of the GNU General Public License along with\n',
 '# this program. If not, see <http://www.gnu.org/licenses/>. \n',
 '# = = = = = = = = = = = = 